In [ ]:
# num 업데이트하면 새로운 법안을 크롤링해서 proposals.sql에 저장

import sqlite3, requests, re
from bs4 import BeautifulSoup

num=0  # Update num to the last bill number

# Create SQL DB
conn = sqlite3.connect('/content/drive/MyDrive/proposals.sql')
curs = conn.cursor()

curs.execute("DROP TABLE proposals;")

# Create a table and its schema
curs.execute("CREATE TABLE IF NOT EXISTS proposals ("
             "bill_num TEXT PRIMARY KEY, title TEXT, issue_date TEXT, head_proposer TEXT, contents TEXT, committee TEXT, process TEXT, proposers TEXT"
             ");")

# Find a tag
def find_tag(tag):
  cont = soup.find_all(tag)
  txt = []
  for paragraph in cont:
    txt.append(paragraph.get_text(" ",strip=True))
  return txt

# Find a class
def find_cls(cls, tag):
  cont = soup.find_all(attrs={cls:tag})
  txt = []
  for paragraph in cont:
    txt.append(paragraph.get_text(" ",strip=True))
  return txt

# Crawl bills
while True:
  try:
    num += 1
    num_str = str(num).zfill(5)
    url = "http://watch.peoplepower21.org/?mid=LawInfo&bill_no=21{}".format(num_str)

    req = requests.get(url, verify=False)
    soup = BeautifulSoup(req.content, "html.parser")
    
    tt = find_tag('h1')[0]
    title = tt.split("(",1)[0]
    head_proposer = tt.split("(",1)[1].split("의원")[0]
    issue_date = ''
    # issue_date = find_cls('class', 'footnote')[0][:10]
    try:
      contents_raw = find_cls('class', 'textType02')[0]
    except:
      contents = ''
    contents = re.sub('[^가-힝一-龥0-9a-zA-Z\\s]', '', contents_raw)
    committee = find_tag('h4')[1]
    process = find_cls('class', 'on')[0]
    proposers = find_cls('class', 'col-sm-8')[0]
    
    # Insert or replace into proposals table
    to_db = [num_str, title, issue_date, head_proposer, contents, committee, process, proposers]
    curs.execute("INSERT OR REPLACE INTO proposals (bill_num, title, issue_date, head_proposer, contents, committee, process, proposers)"
                  "VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
  except:
    print(num-1)
    break

conn.commit()
conn.close()

5


In [ ]:
# 뽑아내기 
import csv, sqlite3

wk_dir = '/content/drive/MyDrive/'

# SQL DB 접근
conn = sqlite3.connect('{0}proposals.sql'.format(wk_dir))
curs = conn.cursor()

# column 이름 나열
curs.execute('PRAGMA TABLE_INFO(proposals)')
labels = [tup[1] for tup in curs.fetchall()]

# '미래차', '전기차', '친환경차', '자동차', '버스', '트럭' 관련 법안 최근 200개만 추출
table = "SELECT * FROM proposals LIMIT 20;"
# table = "SELECT * FROM proposals WHERE contents LIKE '%미래차%' OR contents LIKE '%전기차%' OR contents LIKE '%친환경차%' OR contents LIKE '%자동차%' OR contents LIKE '%버스%' OR contents LIKE '%트럭%' ORDER BY bill_num DESC LIMIT 200;"
curs.execute(table)
dt = curs.fetchall()

result = []
print(labels)
for line in dt:
  result.append(line)
  print(line)

with open('{0}proposal_list.csv'.format(wk_dir), 'w', encoding='utf-8', newline='') as query_csv:
    writer = csv.writer(query_csv)
    writer.writerow(labels)
    writer.writerows(dt)

['bill_num', 'title', 'issue_date', 'head_proposer', 'contents', 'committee', 'process', 'proposers']
('00001', '공공기관의 사회적 가치 실현에 관한 기본법안', '2020. 6. 1.', '박광온', '제안이유 세계적으로 양극화 및 불균형이라는 심각한 위기에 직면하고 있는 상황에서 개인의 노력만으로는 해결할 수 없는 사회문제에 대한 해결책으로 공동체적 가치를 지향하는 사회적 가치의 중요성이 부각되고 있으며 EU와 영국 등 주요 국가에서는 입법화를 통해 사회적 가치를 체계화하고 사회 전반에 이를 확산시키려는 노력을 전개하고 있음 우리의 경우 사회적 경제 기업의 사회적 책임Corporate Social Responsibility 공유가치창출Creating Shared Value 등 사회적 가치 실현과 관련된 다양한 활동이 증가하고 있으나 공공기관의 공공성에 대한 국내외 평가는 낮은 수준이며 공공기관에 대한 국민적 신뢰도 미흡한 상황임 최근 코로나19가 불러온 위기는 우리사회 구조와 제도 전반에 대한 성찰을 요구하고 있고 이윤과 효율이 아니라 사람의 가치 공동체의 가치를 지향하는 방향으로 사회가 대전환해야 한다는 교훈을 주고 있음 사회적 가치는 인권 안전 환경 사회적 약자 배려 양질의 일자리 창출 상생협력 사회통합 등 공공의 이익과 공동체 발전에 기여하는 가치로서 포스트 코로나 시대에 추구해야 할 핵심적 가치임 이에 사회적 가치 실현을 공공부문의 핵심 운영원리로 삼고 업무수행 시 이를 체계적으로 실행하도록 함으로써 공공부문부터 사회적 가치 실현을 선도하고 나아가 민간부문으로 확산시키기 위해 기본법을 제정하려는 것임 이 법은 공공기관이 수행하는 조달 개발 위탁 기타 민간지원 사업에 있어서 비용절감이나 효율성만을 중시하기보다는 사회적 가치를 고려하도록 하고 이러한 사회적 가치의 실현을 공공기관의 평가에 반영토록 하여 우리사회 전반의 공공성을 획기적으로 높이는 것을 목적으로 함 주요

In [ ]:
# Update issue_date

import requests, sqlite3
from bs4 import BeautifulSoup

wk_dir = '/content/drive/MyDrive/'

# SQL DB 접근
conn = sqlite3.connect('{0}proposals.sql'.format(wk_dir))
curs = conn.cursor()

# Find a tag
def find_tag(tag):
  cont = soup.find_all(tag)
  txt = []
  for paragraph in cont:
    txt.append(paragraph.get_text(" ",strip=True))
  return txt

# Find a class
def find_cls(cls, tag):
  cont = soup.find_all(attrs={cls:tag})
  txt = []
  for paragraph in cont:
    txt.append(paragraph.get_text(" ",strip=True))
  return txt

# Crawl bills
for num in range(1,10000):
  num_str = str(num).zfill(5)
  url = "https://www.lawmaking.go.kr/opnPtcp/nsmLmSts/out/21{}/detailRP".format(num_str)

  req = requests.get(url, verify=False)
  soup = BeautifulSoup(req.content, "html.parser")
  try:
    issue_date = find_cls('id', 'pop_wrap')[0].split("호(")[1][:11]
  except:
    issue_date = ''

  # Update issue_date column in proposals table
  curs.execute("UPDATE proposals SET issue_date = '{0}' WHERE bill_num = '{1}';".format(issue_date, num_str))

conn.commit()
conn.close()

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

#아래는 시험용#


https://kimcoder.tistory.com/171

+ URL로 HTML 가져와서파싱하기
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

url="http://www.naver.com/"
html = urlopen(url).read()

soup = BeautifulSoup(html, 'html.parser')

rank = soup.find("dl", id="ranklist")     # dl tag, id=ranklist search.

for i in rank.find_all("li", value=True, id=False):          # li tag, value exist, id no exist. search
     print( i.get_text(" ", strip=True) )  # 문자열을 가져오는데, 태그는 공백으로 두고, 앞뒤 공백 제거.


+헤더 추가
req = Request(url)
req.add_header('User-Agent', 'Mozilla/5.0')
html = urlopen(req).read()



+BeautifulSoup html 서치.

-모든 태그 검색
html_as=soup.find_all("a")     # 모든 a 태그 검색. 접근시 html_as[0], [1], .. 첫번째가 0인덱스.
soup("a")     # 상동

-스트링이 있는 title 태그 모두 검색
soup.title.find_all(string=True)
soup.title(string=True)

-a태그 두 개만 가져옴.
soup.find_all("a", limit=2)

-스트링 검색
soup.find_all(string="elsie")   해당 스트링 검색
 or검색을 하려면 array 로 입력 ["aa", "bb"]
 정규식을 쓰려면 re.compile("정규식")

-태그와 속성(클래스)값으로 검색
soup.find_all("p", "title")
soup.select('p[class="title"]')
ex) <p class="title"></p>

-태그와 태그 사이 찾기
soup.find_all(["a", "b"])

-속성(클래스)값 가져오기
soup.p['class']
soup.p['id']

-보기 좋게 출력
soup.b.prettify()


-검색
soup.body.b     각 첫번째 노드
soup.a     # 처음으로 나오는 a태그

-태그와 클래스명 검색
soup.find_all("a", class_="sister")     # class_ 를 사용. 예약어라서 _를 사용.

-태그와 ID로 찾기
soup.find("div", id="articlebody")
soup.find("div", { "id":"articlebody" } )
soup.find(id="articlebody")

soup.select("#articlebody") ; id로 검색.
soup.select("div#articlebody") ; 태그와 id로 검색.

-태그와 클래스로 찾기 
soup.select('div ol[class="list1"]')

find_all 이나 select는 array로 리턴. 복수개!
하나만 찾을 때는 find 사용.


-검색결과 없으면 None

-태그의 이름 얻기
soup.find("div").name

-속성 얻기
soup.find("div")['class']          없으면 에러
soup.find("div").get('class')     없으면 None


-태그 사이에 있는 중간의 텍스트 얻기
contents 속성 사용.
<p> aaa
<b> one </b>
cc
</p>

soup.b.string          ==> one
soup.b.contents[0]     ==> one

soup.p.contents     ==> aaa, <b>one</b>, cc
     원하는 원소 인덱스를 사용.

-다음 형제 태그
soup.p.next_sibling

-태그 내부 추적 검색
ptag = soup.p
ptag('table')[0]('tr')[0]('td')[0]

-태그 사이 텍스트 전체 
ptag.get_text()




+ HTML 파일에서 읽기
import codecs

page_html = codecs.open('a.html', 'r', 'utf-8')
page_soup = soup(page_html, "html.parser")

+태그 추적하기 
예)

// span 태그의 아이디로 먼저 검색하고, 자식중에 2번째 table 태그를 찾음.
testwebsite_container = page_soup.find("span", id="MainContent2_ctl00_lblContent").findAll("table")[1]

skiptrcnt=1 # skip first tr block
for i,record in enumerate(testwebsite_container.findAll('tr')):    # 모든 tr태그를 검색.
    if skiptrcnt>i:
        continue
    # 처음 n개는 스킵한다.
tnum = record('td')[0].text        # 첫 번째 td의 텍스트
desc = record('td')[1].text
doclink = record('td')[2].text    #세번째 td의 텍스트 (a link의 보이는 링크명으로 보이는 부분이 나옴.)
alink = record('td')[2].find("a")       # 세번째  td에서 하위에 a태그를 찾음.
if alink :
    doclinkurl=testwebsite+alink['href']    # 속성! href의 값을 가져온다. 
closingdate = record('td')[3].text

detail = record('td')[4].text            # td태그 내부에 br태그가 있으면 줄바뀜이 생김.
detail = detail.replace('\n', '')        # 줄바뀜 제거. 


출처: https://crazyj.tistory.com/80 [크레이지J의 탐구생활]

In [ ]:
import sqlite3, requests
from bs4 import BeautifulSoup

# Create SQL DB
conn = sqlite3.connect('/content/drive/MyDrive/proposal_list.sql')
curs = conn.cursor()

# Create a table and its schema
curs.execute("CREATE TABLE IF NOT EXISTS proposals ("
             "bill_num TEXT PRIMARY KEY, title TEXT, head_proposer TEXT, contents TEXT, committee TEXT, process TEXT, proposers TEXT"
             ");")

# Find a tag
def find_tag(tag):
  cont = soup.find_all(tag)
  txt = []
  for paragraph in cont:
    txt.append(paragraph.get_text(" ",strip=True))
  return txt

# Find a class
def find_cls(cls, tag):
  cont = soup.find_all(attrs={cls:tag})
  txt = []
  for paragraph in cont:
    txt.append(paragraph.get_text(" ",strip=True))
  return txt

for num in range(1,10000):
  num_str = str(num).zfill(5)
  url = "http://watch.peoplepower21.org/?mid=LawInfo&bill_no=21{}".format(num_str)

  req = requests.get(url, verify=False)
  soup = BeautifulSoup(req.content, "html.parser")
  
  tt = find_tag('h1')[0]
  title = tt.split("(",1)[0]
  head_proposer = tt.split("(",1)[1].split("의원 ")[0]
  try:
    contents = find_cls('class', 'textType02')[0]
  except:
    contents = ''
  committee = find_tag('h4')[1]
  process = find_cls('class', 'on')[0]
  proposers = find_cls('class', 'col-sm-8')[0]
  
  to_db = [num_str, title, head_proposer, contents, committee, process, proposers]
  print(to_db)
  curs.execute("INSERT OR REPLACE INTO proposals (bill_num, title, head_proposer, contents, committee, process, proposers)"
                "VALUES (?, ?, ?, ?, ?, ?, ?);", to_db)

conn.commit()
conn.close()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:

# 임시 테이블 청소하기
clr = "DROP TABLE IF EXISTS combined;"
curs.execute(clr)
clr = "DROP TABLE IF EXISTS top1;"
curs.execute(clr)
clr = "DROP TABLE IF EXISTS top2;"
curs.execute(clr)
clr = "DROP TABLE IF EXISTS copied;"
curs.execute(clr)

# 전년도와 당해년도 상위 25개그룹(당해기준)의 연간 판매합계를 combined 테이블에 조인
top25 = "CREATE TABLE top1 AS SELECT group_name, (S1+S2+S3+S4+S5+S6+S7+S8+S9+S10+S11+S12) " \
        "FROM (SELECT nation, group_name, company_n, company, " \
        "make_brand, type, segment, model, year, SUM(M1) AS S1, SUM(M2) AS S2, SUM(M3) AS S3, SUM(M4) AS S4, " \
        "SUM(M5) AS S5, SUM(M6) AS S6, SUM(M7) AS S7, SUM(M8) AS S8, SUM(M9) AS S9, SUM(M10) AS S10, SUM(M11) AS S11, " \
        "SUM(M12) AS S12 FROM cars2 WHERE year = {0} AND group_name IS NOT NULL GROUP BY group_name ORDER BY " \
        "SUM(M1+M2+M3+M4+M5+M6+M7+M8+M9+M10+M11+M12) DESC LIMIT 25) ORDER BY nation;".format(year - 1)
curs.execute(top25)

top25 = "CREATE TABLE top2 AS SELECT nation, group_name, (S1+S2+S3+S4+S5+S6+S7+S8+S9+S10+S11+S12) " \
        "FROM (SELECT nation, group_name, company_n, company, " \
        "make_brand, type, segment, model, year, SUM(M1) AS S1, SUM(M2) AS S2, SUM(M3) AS S3, SUM(M4) AS S4, " \
        "SUM(M5) AS S5, SUM(M6) AS S6, SUM(M7) AS S7, SUM(M8) AS S8, SUM(M9) AS S9, SUM(M10) AS S10, SUM(M11) AS S11, " \
        "SUM(M12) AS S12 FROM cars2 WHERE year = {0} AND group_name IS NOT NULL GROUP BY group_name ORDER BY " \
        "SUM(M1+M2+M3+M4+M5+M6+M7+M8+M9+M10+M11+M12) DESC LIMIT 25) ORDER BY nation;".format(year)
curs.execute(top25)

top25 = "CREATE TABLE combined AS SELECT * FROM top2 LEFT JOIN top1 on top2.group_name = top1.group_name;"
curs.execute(top25)

# 2008년까지(최초 데이터) 각 해의 상위 25개그룹(당해기준)의 연간 판매합계를 combined 테이블에 조인
cursor_year = year - 2
while cursor_year > 2007:
    clr = "DROP TABLE IF EXISTS top1;"
    curs.execute(clr)

    top25 = "CREATE TABLE top1 AS SELECT group_name, (S1+S2+S3+S4+S5+S6+S7+S8+S9+S10+S11+S12) " \
            "FROM (SELECT nation, group_name, company_n, company, " \
            "make_brand, type, segment, model, year, SUM(M1) AS S1, SUM(M2) AS S2, SUM(M3) AS S3, SUM(M4) AS S4, " \
            "SUM(M5) AS S5, SUM(M6) AS S6, SUM(M7) AS S7, SUM(M8) AS S8, SUM(M9) AS S9, SUM(M10) AS S10, SUM(M11) AS S11, " \
            "SUM(M12) AS S12 FROM cars2 WHERE year = {0} AND group_name IS NOT NULL GROUP BY group_name ORDER BY " \
            "SUM(M1+M2+M3+M4+M5+M6+M7+M8+M9+M10+M11+M12) DESC LIMIT 25) ORDER BY nation;".format(cursor_year)
    curs.execute(top25)

    # combined 테이블에 overwrite이 불가능하므로 임시테이블 copied 생성
    overwrite = "CREATE TABLE copied AS SELECT * FROM combined;"
    curs.execute(overwrite)

    clr = "DROP TABLE combined;"
    curs.execute(clr)

    top25 = "CREATE TABLE combined AS SELECT * FROM copied LEFT JOIN top1 on copied.group_name = top1.group_name;"
    curs.execute(top25)

    clr = "DROP TABLE copied;"
    curs.execute(clr)

    cursor_year -= 1

top25 = "SELECT * FROM combined;"
curs.execute(top25)
rslt = curs.fetchall()

# 불필요한 열 건너뛰고 데이터만 저장 => 이거 나중에 reverse slicing으로 순서대로 나오게 뽑기. header도. row_list[0,1,-2]
new_result = []
for row in rslt:
    row_list = list(row)
    row = row_list[:2] + row_list[2::2]
    new_result.append(row)

# Header 달기
num_year = len(new_result[0])-2
header1 = ['nation','group_name']
for y in range(num_year):
    header1.append('{0}'.format(year-y))

# 국별 판매량
b_nation = "SELECT nation, year, (SUM(M1)+SUM(M2)+SUM(M3)+SUM(M4)+SUM(M5)+SUM(M6)+SUM(M7)+SUM(M8)+SUM(M9)+SUM(M10) " \
           "+SUM(M11)+SUM(M12)) AS total FROM cars2 GROUP BY year, nation ORDER BY year DESC, nation ASC;"
curs.execute(b_nation)
by_nation = curs.fetchall()

# 행으로 나열된 year를 열로 옮기기
nation_list = []
china = ['china']
europe = ['europe']
japan = ['japan']
korea = ['korea']
malaysia = ['malaysia']
# 말레이시아는 2009년부터 2015년까지만 데이터가 존재
if num_year > 8:
    for y in range(num_year-8):
        malaysia.append('')
us = ['us']
for row in by_nation:
    if row[0] == 'china':
        china.append(row[2])
    elif row[0] == 'europe':
        europe.append(row[2])
    elif row[0] == 'japan':
        japan.append(row[2])
    elif row[0] == 'korea':
        korea.append(row[2])
    elif row[0] == 'malaysia':
        malaysia.append(row[2])
    elif row[0] == 'us':
        us.append(row[2])
nation_list = [china, europe, japan, korea, malaysia, us]

# Header 달기
header2 = ['nation']
for y in range(num_year):
    header2.append('{0}'.format(year-y))

# 차종별 판매량
b_type = "SELECT type, year, (SUM(M1)+SUM(M2)+SUM(M3)+SUM(M4)+SUM(M5)+SUM(M6)+SUM(M7)+SUM(M8)+SUM(M9)+SUM(M10)" \
         "+SUM(M11)+SUM(M12)) AS total FROM cars2 GROUP BY year, type ORDER BY year DESC, type ASC;"
curs.execute(b_type)
by_type = curs.fetchall()

# 행으로 나열된 year를 열로 옮기기
type_list = []
Sedan_Hatchback = ['Sedan/Hatchback']
SUV = ['SUV']
MPV = ['MPV']
Mini_Van = ['Mini Van']

for row in by_type:
    if row[0] == 'Sedan/Hatchback':
        Sedan_Hatchback.append(row[2])
    elif row[0] == 'SUV':
        SUV.append(row[2])
    elif row[0] == 'MPV':
        MPV.append(row[2])
    elif row[0] == 'Mini Van':
        Mini_Van.append(row[2])
type_list = [Sedan_Hatchback, SUV, MPV, Mini_Van]

# Header 달기
header3 = ['type']
for y in range(num_year):
    header3.append('{0}'.format(year-y))

# 쿼리 저장
with open('{0}china_car_yearly_table.csv'.format(wk_dir), 'w', encoding='utf-8', newline='') as table_csv:
    writer = csv.writer(table_csv)
    writer.writerow(header1)
    writer.writerows(new_result)
    writer.writerow('\n')
    writer.writerow(header2)
    writer.writerows(nation_list)
    writer.writerow('\n')
    writer.writerow(header3)
    writer.writerows(type_list)

In [ ]:
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# session = requests.Session()
# retry = Retry(connect=3, backoff_factor=0.5)
# adapter = HTTPAdapter(max_retries=retry)
# session.mount('http://', adapter)
# session.mount('https://', adapter)

# session.get(url)

# ns_session = 21
# bill_num = 1  # 0을 포함한 5자리여야 함. iterate 해야 함
# url = "https://www.lawmaking.go.kr/opnPtcp/nsmLmSts/out/{0}{1}/detailRP".format(ns_session, bill_num)  나중에 API 받아서 처리할 것

url1 = "https://opinion.lawmaking.go.kr/gcom/nsmLmSts/out/2110132/detailRP"
url2 = "http://watch.peoplepower21.org/?mid=LawInfo&bill_no=2109229"

def select_tag(tag):
  cont = soup.find_all(tag)
  txt = []
  for paragraph in cont:
    txt.append(paragraph.get_text(" ",strip=True))
  return txt

def find_tag(cls, tag):
  cont = soup.find_all(attrs={cls:tag})
  txt = []
  for paragraph in cont:
    txt.append(paragraph.get_text(" ",strip=True))
  return txt

for num in range(1,10000):
  num_str = str(num).config/zfill(5)
  url = "http://watch.peoplepower21.org/?mid=LawInfo&bill_no=21{}".format(num_str)

  req = requests.get(url, verify=False)
  soup = BeautifulSoup(req.content, "html.parser")
  
  title = select_tag('h1')[0]
  contents = find_tag('class', 'textType02')[0]
  committee = select_tag('h4')[1]
  process = find_tag('class', 'on')[0]
  proposers = find_tag('class', 'col-sm-8')[0]

  
  to_db = [num_str, title, contents, committee, process, proposers]
  curs.execute("INSERT INTO proposals (bill_num, title, contents, committee, process, proposers")"
                "VALUES (?, ?, ?, ?, ?, ?);", to_db)

conn.commit()
conn.close()

  # print('Title@ {0}\nContents@ {1}\nCommittee@ {2}\nProcess@ {3}\nProposers@ {4}\n'.format(select_tag('h1')[0], find_tag('class', 'textType02')[0], select_tag('h4')[1], find_tag('class', 'on')[0], find_tag('class', 'col-sm-8')[0]))





http://watch.peoplepower21.org/?mid=LawInfo&bill_no=2100001
Title@ 공공기관의 사회적 가치 실현에 관한 기본법안(박광온의원 등 16인)
Contents@ 제안이유 세계적으로 양극화 및 불균형이라는 심각한 위기에 직면하고 있는 상황에서 개인의 노력만으로는 해결할 수 없는 사회문제에 대한 해결책으로 공동체적 가치를 지향하는 사회적 가치의 중요성이 부각되고 있으며, EU와 영국 등 주요 국가에서는 입법화를 통해 사회적 가치를 체계화하고 사회 전반에 이를 확산시키려는 노력을 전개하고 있음. 우리의 경우 사회적 경제, 기업의 사회적 책임(Corporate Social Responsibility), 공유가치창출(Creating Shared Value) 등 사회적 가치 실현과 관련된 다양한 활동이 증가하고 있으나, 공공기관의 공공성에 대한 국내외 평가는 낮은 수준이며, 공공기관에 대한 국민적 신뢰도 미흡한 상황임. 최근 코로나19가 불러온 위기는 우리사회 구조와 제도 전반에 대한 성찰을 요구하고 있고, 이윤과 효율이 아니라 사람의 가치, 공동체의 가치를 지향하는 방향으로 사회가 대전환해야 한다는 교훈을 주고 있음. 사회적 가치는 인권, 안전, 환경, 사회적 약자 배려, 양질의 일자리 창출, 상생협력, 사회통합 등 공공의 이익과 공동체 발전에 기여하는 가치로서 포스트 코로나 시대에 추구해야 할 핵심적 가치임. 이에 사회적 가치 실현을 공공부문의 핵심 운영원리로 삼고, 업무수행 시 이를 체계적으로 실행하도록 함으로써 공공부문부터 사회적 가치 실현을 선도하고 나아가 민간부문으로 확산시키기 위해 기본법을 제정하려는 것임. 이 법은 공공기관이 수행하는 조달, 개발, 위탁, 기타 민간지원 사업에 있어서 비용절감이나 효율성만을 중시하기보다는 사회적 가치를 고려하도록 하고, 이러한 사회적 가치의 실현을 공공기관의 평가에 반영토록 하여, 우리사회 전반의 공공성을 획기적으로 높이는 것을 목적으로 함. 주요내용 가. 이 법은 공공부문의 사회

IndexError: ignored

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

contents = 'pre'

html = urlopen("https://www.lawmaking.go.kr/opnPtcp/nsmLmSts/out/2100002/detailRP")
bs = BeautifulSoup(html.read(), "html.parser")
txt = bs.find("div", {"class": contents}) # 가져올 태그 검색
print(txt) # 선택한 html 출력하기
print(txt.text) # html에서 불필요한 테크 제외하고 출력하기

None


AttributeError: ignored

In [ ]:
print(html.datahandler)

AttributeError: ignored